# 自動去除圖像中特定類別系統

1. 說明：此系統支持輸入中出現「人、鳥、狗、貓」之圖像，可將偵測到之「人、鳥、狗、貓」從圖像中移除。
2. 輸入：一有含「人、鳥、狗、貓」至少一類別之圖像。故預計產生四個分類，「人」、「鳥」、「狗」、「貓」。
3. 輸出：一將輸入圖中有「人、鳥、狗、貓」至少一類別之目標移除後之圖像。
4. 實現流程：
![](https://imgur.com/vtdM3MS.png)




5. 實作步驟：
*  使用 wget 指令將必要 zip 檔 M11115017_colab.zip（共 2.4 GB）下載至本地端



In [1]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1MhILX0Ke80-LdMJhblbjK9ZaKF-5hNez' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1MhILX0Ke80-LdMJhblbjK9ZaKF-5hNez" -O M11115017_colab.zip && rm -rf /tmp/cookies.txt

--2023-06-05 07:05:39--  https://docs.google.com/uc?export=download&confirm=t&id=1MhILX0Ke80-LdMJhblbjK9ZaKF-5hNez
Resolving docs.google.com (docs.google.com)... 142.250.101.101, 142.250.101.102, 142.250.101.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.101.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-b0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uuhvte4g7fk6to5deeb3sdpgse4gq3p8/1685948700000/04397771258705137741/*/1MhILX0Ke80-LdMJhblbjK9ZaKF-5hNez?e=download&uuid=e90a86a1-ee4e-4820-aee1-24a2a4f7f78b [following]
--2023-06-05 07:05:39--  https://doc-0g-b0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uuhvte4g7fk6to5deeb3sdpgse4gq3p8/1685948700000/04397771258705137741/*/1MhILX0Ke80-LdMJhblbjK9ZaKF-5hNez?e=download&uuid=e90a86a1-ee4e-4820-aee1-24a2a4f7f78b
Resolving doc-0g-b0-docs.googleusercontent.com (doc-0g-b0-docs.googleusercontent.com)... 142.250.141.

*  解壓縮 M11115017_colab.zip

In [2]:
!unzip /content/M11115017_colab.zip

串流輸出內容已截斷至最後 5000 行。
  inflating: workspace/lama/bin/ultralytics/dataset/labels/train/000000132533.txt  
  inflating: __MACOSX/workspace/lama/bin/ultralytics/dataset/labels/train/._000000132533.txt  
  inflating: workspace/lama/bin/ultralytics/dataset/labels/train/000000574961.txt  
  inflating: __MACOSX/workspace/lama/bin/ultralytics/dataset/labels/train/._000000574961.txt  
  inflating: workspace/lama/bin/ultralytics/dataset/labels/train/000000033177.txt  
  inflating: __MACOSX/workspace/lama/bin/ultralytics/dataset/labels/train/._000000033177.txt  
  inflating: workspace/lama/bin/ultralytics/dataset/labels/train/000000327804.txt  
  inflating: __MACOSX/workspace/lama/bin/ultralytics/dataset/labels/train/._000000327804.txt  
  inflating: workspace/lama/bin/ultralytics/dataset/labels/train/000000157397.txt  
  inflating: __MACOSX/workspace/lama/bin/ultralytics/dataset/labels/train/._000000157397.txt  
  inflating: workspace/lama/bin/ultralytics/dataset/labels/train/000000524830.txt  


* 切換至 ultralytics 資料夾，下載其使用時需使用之工具

In [3]:
%cd /content/workspace/lama/ultralytics
!pip install -r requirements.txt 

/content/workspace/lama/ultralytics
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


* 下載使用 lama 時需使用之工具



In [4]:
!pip install pytorch_lightning
!pip install kornia==0.5.0
!pip install hydra-core --upgrade
!pip install webdataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.5/271.5 kB 8.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 5.3 MB/s eta 0:00:00


* 使用下方指令即可使用本系統，須改變之地方為圖片路徑 **infer_pic**，應改為使用者須套入之圖片路徑，此處圖片使用 /content/workspace/lama/sample_images 中之測試影像 sample1.png
* 程式執行後可於 /content/workspace/lama/output 路徑領取最終執行結果圖（detect_mask.png）



In [5]:
%cd /content/workspace/lama
!python bin/predict.py refine=False device=cpu model.path=$(pwd)/big-lama  indir=$(pwd)/test_image  outdir=$(pwd)/output yolo_model=/content/workspace/lama/ultralytics/runs/detect/train/weights/best.pt infer_pic=/content/workspace/lama/sample_images/sample1.png

/content/workspace/lama
Detectron v2 is not installed
/content/workspace/lama/bin/predict.py:43: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path='../configs/prediction', config_name='default.yaml')
/usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(

here
image 1/1 /content/workspace/lama/sample_images/sample1.png: 640x640 1 person, 1558.2ms
Speed: 16.7ms preprocess, 1558.2ms inference, 25.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/workspace/lama/runs/detect/predict
[2023-06-05 07:13:02,187][saicinpainting.utils][WARNING] - Setting signal 10 handler <function print_traceback_handler at 0x7f3b